# Entrenamiento de clasificadores con imágenes sintéticas y evaluación del rendimiento con conjunto de test. 

# Métrica *Classification Accuracy Score*

En el presente Notebook, se realiza el entrenamiento de clasificadores con imágenes sintéticas y se evalúa el rendimiento de los mismos con un conjunto de test, siguiendo la filosofía de la métrica *Classification Accuracy Score*, la cual considera que si un modelo generativo genera imágenes de buena calidad, deberían ser buenas para entrenar cualquier clasificador, y que dicho clasificador debería obtener buenas métricas de evaluación. 

## Librerías necesarias

In [1]:
import os
import random

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import torch.optim as optim

from tqdm import tqdm

import cv2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV


## Parámetros importantes

In [2]:
IMAGE_SIZE = 64
PIC_CHANNELS = 3
PIC_DIMENSION = 2
BATCH_SIZE = 32
EPOCHS = 75
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps")

## Carga de las imágenes sintéticas y reales

### Datos reales

Cargamos las imágenes reales de lesiones de piel, el mismo número para cada clase. De este conjunto total, extraemos una parte para test de los clasificadores que se entrenarán.

In [3]:
DATA_DIR = '/Users/alzorrcarri/skin_lesion_training_images'
real_dataset = []
name_files = os.listdir(DATA_DIR)

for name in name_files:
    if name.endswith('.jpg'):
        img = torch.tensor(plt.imread(os.path.join(DATA_DIR, name)), dtype=torch.float32)
        img = img.permute(2, 0, 1)
        real_dataset.append({'image': img/255.0, 'label': 1})

In [4]:
DATA_DIR = '/Users/alzorrcarri/train_images_benign'

name_files = os.listdir(DATA_DIR)

for name in name_files:
    if name.endswith('.jpg'):
        img = torch.tensor(plt.imread(os.path.join(DATA_DIR, name)), dtype=torch.float32)
        img = img.permute(2, 0, 1)
        real_dataset.append({'image': img/255.0, 'label': 0})

In [5]:
random.shuffle(real_dataset)

In [6]:
len(real_dataset)

12602

In [6]:
test_dataset = real_dataset[:len(real_dataset)//7]

In [23]:
len(test_dataset)

1800

### Datos sintéticos

Cargamos el total de imágenes sintéticas de ambos tipos de lesiones de piel generados con los DDPMs incondicionales entrenados. 

In [7]:
DATA_DIR = '/Users/alzorrcarri/Library/CloudStorage/Dropbox/tfm/codigo/Métricas de evaluación/malignant/synthetic_images'

synthetic_dataset = []
name_files = os.listdir(DATA_DIR)

for name in name_files:
    if name.endswith('.jpg'):
        img = torch.tensor(plt.imread(os.path.join(DATA_DIR, name)), dtype=torch.float32)
        img = img.permute(2, 0, 1)
        synthetic_dataset.append({'image': img/255.0, 'label': 1})

In [8]:
DATA_DIR = '/Users/alzorrcarri/Library/CloudStorage/Dropbox/tfm/codigo/Métricas de evaluación/benign/synthetic_images'

name_files = os.listdir(DATA_DIR)

for name in name_files:
    if name.endswith('.jpg'):
        img = torch.tensor(plt.imread(os.path.join(DATA_DIR, name)), dtype=torch.float32)
        img = img.permute(2, 0, 1)
        synthetic_dataset.append({'image': img/255.0, 'label': 0})

In [9]:
random.shuffle(synthetic_dataset)
len(synthetic_dataset)

1000

## Descriptor HOG + Clasificador SVM

Vamos a usar dos tipos de clasificadores. Por un lado, vamos a tomar un descriptor (en este caso HOG), que extrae ciertas características de las imágenes de entrenamiento, y son con las que vamos a entrenar un clasificador SVM. Usaremos un enfoque de GridSearch para encontrar los mejores parámetros del mismo. El rendimiento del clasificador SVM lo pondremos a prueba con las imágenes de test, sobre las que también se aplicará el descriptor HOG.

*Nota: En este caso, no se puede normalizar las imágenes al cargarlas, ya que el descriptor HOG no funciona con imágenes normalizadas. Tampoco se puede establecer las imágenes como tensores de tipo float, sino que deben ser de tipo uint8.*

In [41]:
# Cargamos el descriptor HOG de OpenCV y lo aplicamos a las imágenes generadas
win_size = (64, 64)
block_size = (8, 8)
block_stride = (2, 2)
cell_size = (4, 4)
n_bins_orientacion = 9
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, n_bins_orientacion)

In [51]:
# Aplicamos el descriptor HOG a las imágenes generadas
hog_syn_images = []
for data in synthetic_dataset:
    img = data['image'].permute(1, 2, 0).numpy()
    hog_syn_images.append({'image':hog.compute(img).flatten(), 'label':data['label']})

In [52]:
# Aplicamos el descriptor HOG a las imágenes reales
hog_real_images = []
for data in real_dataset:
    img = data['image'].permute(1, 2, 0).numpy()
    hog_real_images.append({'image':hog.compute(img).flatten(), 'label':data['label']})

In [53]:
# Creamos el conjunto de entrenamiento con el descriptor HOG de cada imagen generada y su etiqueta
X_train = []
y_train = []
for data in hog_syn_images:
    X_train.append(data['image'])
    y_train.append(data['label'])

In [54]:
# Buscamos los mejores parámetros para el clasificador SVM mediante GridSearchCV
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = SVC()
clf = GridSearchCV(estimator=svc, param_grid=parameters, scoring='accuracy', cv=5)
clf.fit(X_train, y_train)
best_params = clf.best_params_

In [55]:
svm = SVC(kernel=best_params['kernel'], C=best_params['C'], random_state=0)
svm.fit(X_train, y_train)

SVC(C=10, random_state=0)

In [56]:
X_test = []
y_test = []
for data in hog_real_images[:len(hog_real_images)//7]:
    X_test.append(data['image'])
    y_test.append(data['label'])

In [58]:
pred_test = svm.predict(X_test)

print("Accuracy: ", accuracy_score(test_labels, pred_test))
print("Confusion Matrix: \n", confusion_matrix(test_labels, pred_test))
print("Precision: ", precision_score(test_labels, pred_test))
print("Recall: ", recall_score(test_labels, pred_test))
print("Classification Report: \n", classification_report(test_labels, pred_test))

Accuracy:  0.4911111111111111
Confusion Matrix: 
 [[293 668]
 [248 591]]
Precision:  0.4694201747418586
Recall:  0.7044100119189511
Classification Report: 
               precision    recall  f1-score   support

           0       0.54      0.30      0.39       961
           1       0.47      0.70      0.56       839

    accuracy                           0.49      1800
   macro avg       0.51      0.50      0.48      1800
weighted avg       0.51      0.49      0.47      1800



## Redes neuronales: SkinLesNet

Por otro lado, el otro enfoque de clasificación es con un modelo de red neuronal que ya hemos empleado anteriormente para la extracción de carcaterísticas de las imágenes de lesiones de piel, que es la red SkinLesNet. Dicha red se creó originalmente paratareas de clasificación de lesiones de piel y obtuvo muy buenos resultados (https://www.mdpi.com/2613640). Por ello, creemos que es la arquitectura más oportuna para usar en este caso.

Con las imágenes sintéticas, creamos un dataset de entrenamiento para la red neuronal. Una vez entrenada, pasamos las imágenes de test por la red neuronal y comparamos las etiquetas predichas con las reales, y calculamos distintas métricas de rendimiento del clasificador.

### Primer enfoque: entrenar solo con imágenes sintéticas

Entrenamos el clasificador solo con imágenes sintéticas y evaluamos su rendimiento con las imágenes de test, que son imágenes reales.

In [11]:
train_loader = DataLoader(synthetic_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
class SkinLesNet(nn.Module):
    def __init__(self, IMAGE_SIZE):
        super(SkinLesNet, self).__init__()
        # 1st Convolutional Input Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 2nd Convolutional Input Layer
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        # 3rd Convolutional Layer
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        
        # 4th Convolutional Layer
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        self.dropout1 = nn.Dropout(0.5)

        self.flatten = nn.Flatten()
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * (IMAGE_SIZE // 16) * (IMAGE_SIZE // 16), 64)  # Adjusting for downsampling
        self.dropout2 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 2)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        
        x = self.dropout1(x)
        
        x = self.flatten(x)
        
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = F.softmax(self.fc2(x), dim=1)
        
        return x

In [13]:
skinlesnet = SkinLesNet(IMAGE_SIZE).to(DEVICE)
optimizer = optim.Adam(skinlesnet.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()
lr_sch = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-15, verbose=False)

/opt/anaconda3/envs/tfm/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [16]:
def calculate_accuracy(output, target):
    _, predicted = torch.max(output, 1)
    correct = (predicted == target).sum().item()
    return correct / target.size(0)

In [15]:
for epoch in range(EPOCHS):
    skinlesnet.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    progress_bar.set_description(f"Epoch {epoch}")
    for step, batch in progress_bar:
        images_batch = batch['image'].to(DEVICE)
        labels_batch = batch['label'].to(DEVICE).long()
        optimizer.zero_grad(set_to_none=True)
        output = skinlesnet(images_batch)
        loss_value = loss(output, labels_batch)
        loss_value.backward()
        optimizer.step()
        epoch_loss += loss_value.item()
        epoch_accuracy += calculate_accuracy(output, labels_batch)
        progress_bar.set_postfix(loss=epoch_loss / (step + 1), accuracy=epoch_accuracy / (step + 1))
    lr_sch.step(epoch_loss / len(train_loader))

Epoch 74: 100%|██████████| 32/32 [00:00<00:00, 92.16it/s, accuracy=0.955, loss=0.36] 


In [14]:
# Conjunto de test
test_images = []
test_labels = []
for i in range(len(test_dataset)):
    test_images.append(test_dataset[i]['image'])
    test_labels.append(test_dataset[i]['label'])

test_images = torch.stack(test_images)
test_labels = torch.tensor(test_labels)

In [25]:
# Evaluamos el modelo en el conjunto de test
skinlesnet.eval()
pred_test = skinlesnet(test_images.to(DEVICE)).argmax(dim=1).cpu().numpy()

print("Accuracy: ", accuracy_score(test_labels, pred_test))
print("Confusion Matrix: \n", confusion_matrix(test_labels, pred_test))
print("Precision: ", precision_score(test_labels, pred_test))
print("Recall: ", recall_score(test_labels, pred_test))
print("Classification Report: \n", classification_report(test_labels, pred_test))

Accuracy:  0.7122222222222222
Confusion Matrix: 
 [[660 301]
 [217 622]]
Precision:  0.6738894907908992
Recall:  0.7413587604290822
Classification Report: 
               precision    recall  f1-score   support

           0       0.75      0.69      0.72       961
           1       0.67      0.74      0.71       839

    accuracy                           0.71      1800
   macro avg       0.71      0.71      0.71      1800
weighted avg       0.72      0.71      0.71      1800



### Segundo enfoque: entrenar con imágenes reales y sintéticas

Ahora entrenamos el clasificador con las imágenes reales y sintéticas, y evaluamos su rendimiento con las imágenes de test, que son únicamente reales.

In [10]:
train_dataset = real_dataset[len(real_dataset)//7:] + synthetic_dataset

In [11]:
len(train_dataset)

11802

In [12]:
random.shuffle(train_dataset)

In [13]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [34]:
skinlesnet = SkinLesNet(IMAGE_SIZE).to(DEVICE)
optimizer = optim.Adam(skinlesnet.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()
lr_sch = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-15, verbose=False)

In [35]:
for epoch in range(EPOCHS):
    skinlesnet.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    progress_bar.set_description(f"Epoch {epoch}")
    for step, batch in progress_bar:
        images_batch = batch['image'].to(DEVICE)
        labels_batch = batch['label'].to(DEVICE).long()
        optimizer.zero_grad(set_to_none=True)
        output = skinlesnet(images_batch)
        loss_value = loss(output, labels_batch)
        loss_value.backward()
        optimizer.step()
        epoch_loss += loss_value.item()
        epoch_accuracy += calculate_accuracy(output, labels_batch)
        progress_bar.set_postfix(loss=epoch_loss / (step + 1), accuracy=epoch_accuracy / (step + 1))
    lr_sch.step(epoch_loss / len(train_loader))

Epoch 74: 100%|██████████| 369/369 [00:04<00:00, 89.44it/s, accuracy=0.956, loss=0.357]


In [36]:
# Evaluamos el modelo en el conjunto de test
skinlesnet.eval()
pred_test = skinlesnet(test_images.to(DEVICE)).argmax(dim=1).cpu().numpy()

print("Accuracy: ", accuracy_score(test_labels, pred_test))
print("Confusion Matrix: \n", confusion_matrix(test_labels, pred_test))
print("Precision: ", precision_score(test_labels, pred_test))
print("Recall: ", recall_score(test_labels, pred_test))
print("Classification Report: \n", classification_report(test_labels, pred_test))

Accuracy:  0.8533333333333334
Confusion Matrix: 
 [[838 123]
 [141 698]]
Precision:  0.8501827040194885
Recall:  0.831942789034565
Classification Report: 
               precision    recall  f1-score   support

           0       0.86      0.87      0.86       961
           1       0.85      0.83      0.84       839

    accuracy                           0.85      1800
   macro avg       0.85      0.85      0.85      1800
weighted avg       0.85      0.85      0.85      1800



### Entrenamiento de la red solo con el conjunto de entrenamiento

Finalmente, entrenamos la red únicamente con las imágenes reales y evaluamos su rendimiento con las imágenes de test, para así tener un marco de referencia para comparar con los resultados anteriores.

In [37]:
train_loader = DataLoader(real_dataset[len(real_dataset)//7:], batch_size=BATCH_SIZE, shuffle=True)

In [38]:
skinlesnet = SkinLesNet(IMAGE_SIZE).to(DEVICE)
optimizer = optim.Adam(skinlesnet.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()
lr_sch = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-15, verbose=False)

/opt/anaconda3/envs/tfm/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [39]:
for epoch in range(EPOCHS):
    skinlesnet.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    progress_bar.set_description(f"Epoch {epoch}")
    for step, batch in progress_bar:
        images_batch = batch['image'].to(DEVICE)
        labels_batch = batch['label'].to(DEVICE).long()
        optimizer.zero_grad(set_to_none=True)
        output = skinlesnet(images_batch)
        loss_value = loss(output, labels_batch)
        loss_value.backward()
        optimizer.step()
        epoch_loss += loss_value.item()
        epoch_accuracy += calculate_accuracy(output, labels_batch)
        progress_bar.set_postfix(loss=epoch_loss / (step + 1), accuracy=epoch_accuracy / (step + 1))
    lr_sch.step(epoch_loss / len(train_loader))

Epoch 74: 100%|██████████| 338/338 [00:03<00:00, 85.71it/s, accuracy=0.959, loss=0.355]


In [40]:
# Evaluamos el modelo en el conjunto de test
skinlesnet.eval()
pred_test = skinlesnet(test_images.to(DEVICE)).argmax(dim=1).cpu().numpy()

print("Accuracy: ", accuracy_score(test_labels, pred_test))
print("Confusion Matrix: \n", confusion_matrix(test_labels, pred_test))
print("Precision: ", precision_score(test_labels, pred_test))
print("Recall: ", recall_score(test_labels, pred_test))
print("Classification Report: \n", classification_report(test_labels, pred_test))

Accuracy:  0.86
Confusion Matrix: 
 [[863  98]
 [154 685]]
Precision:  0.8748403575989783
Recall:  0.8164481525625745
Classification Report: 
               precision    recall  f1-score   support

           0       0.85      0.90      0.87       961
           1       0.87      0.82      0.84       839

    accuracy                           0.86      1800
   macro avg       0.86      0.86      0.86      1800
weighted avg       0.86      0.86      0.86      1800



##  ResNet50

In [15]:
from torchvision import models
from torchvision.models import ResNet50_Weights

In [17]:
resnet50_model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

for param in resnet50_model.parameters():
    param.requires_grad = False

In [18]:
class New_ResNet50(nn.Module):
    def __init__(self, base_model):
        super(New_ResNet50, self).__init__()
        # Cargamos el modelo pre-entrenado con los parámetros congelados
        # Cambiamos la aquitectura de la capa fully-connected para que tenga 1024 neuronas
        self.base_model = base_model
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 1024)

        # Descongelamos los parámetros de la capa fully-connected para que se actualicen durante el entrenamiento
        # ya que hemos cambiado su arquitectura
        for param in self.base_model.fc.parameters():
            param.requires_grad = True

        # Añadimos varias capas fully-connected para la clasificación
        # https://link.springer.com/article/10.1007/s00521-022-07793-2#Sec14
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.base_model(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = self.fc5(x)
        return F.softmax(x, dim=1)

In [19]:
resnet50_model = New_ResNet50(resnet50_model).to(DEVICE)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, resnet50_model.parameters()), lr=0.001)
loss = nn.CrossEntropyLoss()
lr_sch = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, 
                                        patience=1, threshold=0.0001, threshold_mode='rel', 
                                        cooldown=0, min_lr=0, eps=1e-15, verbose=False)

/opt/anaconda3/envs/tfm/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [20]:
for epoch in range(EPOCHS):
    resnet50_model.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    progress_bar.set_description(f"Epoch {epoch}")
    for step, batch in progress_bar:
        images_batch = batch['image'].to(DEVICE)
        labels_batch = batch['label'].to(DEVICE).long()
        optimizer.zero_grad(set_to_none=True)
        output = resnet50_model(images_batch)
        loss_value = loss(output, labels_batch)
        loss_value.backward()
        optimizer.step()
        epoch_loss += loss_value.item()
        epoch_accuracy += calculate_accuracy(output, labels_batch)
        progress_bar.set_postfix(loss=epoch_loss / (step + 1), accuracy=epoch_accuracy / (step + 1))
    lr_sch.step(epoch_loss / len(train_loader))

Epoch 74: 100%|██████████| 369/369 [00:09<00:00, 37.76it/s, accuracy=0.949, loss=0.364]


In [23]:
# Evaluamos el modelo en el conjunto de test
resnet50_model.eval()
pred_test = resnet50_model(test_images.to(DEVICE)).argmax(dim=1).cpu().numpy()

print("Accuracy: ", accuracy_score(test_labels, pred_test))
print("Confusion Matrix: \n", confusion_matrix(test_labels, pred_test))
print("Precision: ", precision_score(test_labels, pred_test))
print("Recall: ", recall_score(test_labels, pred_test))
print("Classification Report: \n", classification_report(test_labels, pred_test))

Accuracy:  0.8238888888888889
Confusion Matrix: 
 [[803 170]
 [147 680]]
Precision:  0.8
Recall:  0.8222490931076178
Classification Report: 
               precision    recall  f1-score   support

           0       0.85      0.83      0.84       973
           1       0.80      0.82      0.81       827

    accuracy                           0.82      1800
   macro avg       0.82      0.82      0.82      1800
weighted avg       0.82      0.82      0.82      1800

